# calc plane

In [1]:
import numpy as np
from leicacam import CAM

In [2]:
cam = CAM()

In [3]:
def get_position():
    info = cam.get_information()
    return np.array((float(info['xpos'])*1e6, float(info['ypos'])*1e6, float(info['zpos'])*1e6))

In [8]:
get_position()

array([ 59244.2617,  29479.2964,      0.    ])

In [12]:
p1 = get_position()

In [13]:
p2 = get_position()

In [30]:
p3 = get_position()

In [14]:
p4 = get_position()

```
1            2
3            4
```

In [16]:
p1, p4

(array([ 59349.1054,  29463.691 ,      0.    ]),
 array([ 83150.807 ,  42937.1509,      0.    ]))

In [18]:
dx = p4[0]-p1[0]
dy = p4[1]-p1[1]
dx, dy

(23801.701599999986, 13473.459900000002)

In [19]:
imgsize = 1550 # microns
fielddist = imgsize * 0.90 # overlap
fielddist

1395.0

In [20]:
from IPython import display
resolution = 512
precision = (imgsize) // resolution # integer devision, in microns
display.Math(str(precision) + '\mu m') 

<IPython.core.display.Math object>

In [30]:
xsize = dx + imgsize
ysize = dy + imgsize
xsize, ysize

(25351.701599999986, 15023.459900000002)

In [31]:
xfields = round(xsize / fielddist)
yfields = round(ysize / fielddist)
xfields, yfields

(18.0, 11.0)

In [23]:
cam.socket.close()

# automated scan

In [32]:
from leicaautomator.automator import find_spots
from leicaautomator.utils import save_regions, flatten
from leicaexperiment import Experiment
from leicascanningtemplate import ScanningTemplate
from leicacam import CAM

In [33]:
def waitfor(cam, key, value=None):
    from time import sleep
    while True:
        msgs = cam.receive()
        for msg in msgs:
            if value and msg.get(key) == value:
                return msg
            elif not value and msg.get(key):
                return msg
        sleep(10e-3)

## rotate images

In [34]:
whole = 'experiment--2015_04_08_12_03_12'
exp_path = r"D:\MatrixScreenerImages\3.3.0_9757\ ".strip()
experiment = Experiment(exp_path + whole)
!dir $experiment.path

 Volume in drive D is DATADRIVE1
 Volume Serial Number is F805-2E08

 Directory of D:\MatrixScreenerImages\3.3.0_9757\experiment--2015_04_08_12_03_12

04/08/2015  02:03 PM    <DIR>          .
04/08/2015  02:03 PM    <DIR>          ..
04/08/2015  02:03 PM    <DIR>          AdditionalData
04/08/2015  02:03 PM    <DIR>          slide--S00
               0 File(s)              0 bytes
               4 Dir(s)  1,677,125,873,664 bytes free


In [35]:
compressed = experiment.compress(delete_tif=True)
assert len(compressed) == len(experiment.images)

In [36]:
from PIL import Image
for img in experiment.images:
    i = Image.open(img)
    i = i.rotate(270)
    i.save(img)

In [37]:
experiment.stitch()

['D:\\MatrixScreenerImages\\3.3.0_9757\\experiment--2015_04_08_12_03_12\\stitched--U00--V00--C00--Z00.png']

## find regions

In [38]:
regions = find_spots(experiment)

Well distance x in microns: 1655.71238989
Well distance y in microns: 1669.07906765
Well offsets
( well )   x[um], y[um]
-----------------------
( 0,  1)     -60,   -81
( 0,  2)     -50,   -99
( 1,  0)     -19,   -98
( 1,  1)     -29,  -162
( 1,  2)     -81,  -182
( 1,  3)    -356,  -426
( 1,  4)    -386,  -387
( 1,  5)    -266,  -247
( 1,  6)    -172,  -139
( 1,  7)    -253,  -334
( 1,  8)    -180,  -205
( 2,  0)    -123,  -278
( 2,  1)     -66,  -281
( 2,  2)    -124,  -253
( 2,  3)    -224,  -251
( 2,  4)      10,  -145
( 2,  5)      -2,   -74
( 2,  6)    -133,  -186
( 2,  7)    -145,   -90
( 2,  8)    -135,    -8
( 3,  0)    -119,  -250
( 3,  1)     -50,  -241
( 3,  2)     -18,  -279
( 3,  3)      18,  -406
( 3,  4)     -76,  -183
( 3,  5)     -67,  -269
( 3,  6)     -31,  -341
( 3,  7)    -227,   -61
( 3,  8)    -170,     3
( 4,  0)     -35,  -180
( 4,  1)    -121,  -193
( 4,  2)     -76,  -203
( 4,  3)    -225,  -124
( 4,  4)     -37,  -223
( 4,  5)     -18,  -189
( 4,  6)     -2

In [41]:
labels, regions = regions

In [42]:
save_regions(regions, 'regions.json')

TypeError: <matplotlib.text.Text object at 0x000000000F8FBFD0> is not JSON serializable

In [43]:
for r in regions:
    print(r.well_x, r.well_y, r.real_x, r.real_y)

0 0 0.0593777158817 0.0299091250293
0 1 0.0593177051786 0.0314970109356
0 2 0.059327442745 0.0331485353776
1 0 0.0610142489444 0.0298114554965
1 1 0.0610042523824 0.0314159567221
1 2 0.0609527984535 0.0330648714227
1 3 0.0606773856751 0.0344898935622
1 4 0.0606473317343 0.0361981889251
1 5 0.0607670038132 0.0380077697377
1 6 0.0608612371946 0.0397846296732
1 7 0.0607805239738 0.0412589098275
1 8 0.0608537028064 0.0430563301519
2 0 0.0625662989296 0.0296314395119
2 1 0.0626234576354 0.0312969242947
2 2 0.0625650162426 0.032994299345
2 3 0.0624646428934 0.0346658143182
2 4 0.0626991948312 0.0364400836641
2 5 0.0626873925224 0.0381805236343
2 6 0.0625564049887 0.0397373361175
2 7 0.0625443867749 0.0415021917121
2 8 0.0625545958408 0.0432536819371
3 0 0.0642260215382 0.0296589489601
3 1 0.0642951226786 0.0313374430041
3 2 0.064326413 0.032968067192
3 3 0.0643627008147 0.0345104866506
3 4 0.0642692841056 0.0364026114948
3 5 0.0642775541496 0.0379856587959
3 6 0.0643137410335 0.0395822791777

# scan whole with shg

In [44]:
imgsize = 246 # microns
fielddist = imgsize * 0.90 # overlap
fielddist

221.4

In [46]:
from IPython import display
resolution = 1024
precision = (imgsize*1000) // resolution # integer devision, in microns
display.Math(str(precision) + 'nm') 

<IPython.core.display.Math object>

In [47]:
xsize = dx + imgsize
ysize = dy + imgsize
xsize, ysize

(24047.701599999986, 13719.459900000002)

In [48]:
xfields = round(xsize / fielddist)
yfields = round(ysize / fielddist)
xfields, yfields

(109.0, 62.0)

# compress 17GB

In [49]:
experiment = Experiment(exp_path + 'experiment--2015_04_08_14_20_02')

In [50]:
experiment.compress(delete_tif=True)

['D:\\MatrixScreenerImages\\3.3.0_9757\\experiment--2015_04_08_14_20_02\\slide--S00\\chamber--U00--V00\\field--X00--Y00\\image--L0000--S00--U00--V00--J08--E00--O00--X00--Y00--T0000--Z00--C00.png',
 'D:\\MatrixScreenerImages\\3.3.0_9757\\experiment--2015_04_08_14_20_02\\slide--S00\\chamber--U00--V00\\field--X00--Y00\\image--L0000--S00--U00--V00--J08--E00--O00--X00--Y00--T0000--Z00--C01.png',
 'D:\\MatrixScreenerImages\\3.3.0_9757\\experiment--2015_04_08_14_20_02\\slide--S00\\chamber--U00--V00\\field--X00--Y01\\image--L0000--S00--U00--V00--J08--E00--O00--X00--Y01--T0000--Z00--C00.png',
 'D:\\MatrixScreenerImages\\3.3.0_9757\\experiment--2015_04_08_14_20_02\\slide--S00\\chamber--U00--V00\\field--X00--Y01\\image--L0000--S00--U00--V00--J08--E00--O00--X00--Y01--T0000--Z00--C01.png',
 'D:\\MatrixScreenerImages\\3.3.0_9757\\experiment--2015_04_08_14_20_02\\slide--S00\\chamber--U00--V00\\field--X00--Y02\\image--L0000--S00--U00--V00--J08--E00--O00--X00--Y02--T0000--Z00--C00.png',
 'D:\\MatrixScr

## -> 4GB